## Analyzing Healthy UC Berkeley Dining Hall Meals in Comparison to the Mediterranean Diet

Contributors: Contributors: Richa Bhattacharya, Audrey Booher, Anoushka Gupta, Daysha Patalot, Yinan Xia


In [ ]:
!pip install -r requirements.txt

In [11]:
import numpy as np
import pandas as pd
import http.client

In [12]:
apikey = "an9JzugedN7gb0EvKlgsVs2iMVLIFVYOLcpdqUqo"  # Replace with a real key!  "DEMO_KEY" will be slow...

In [13]:
import fooddatacentral as fdc

fdc.search(apikey,"crunchy peanut butter")

,fdcId,description,lowercaseDescription,dataType,gtinUpc,publishedDate,brandOwner,brandName,ingredients,marketCountry,...,foodNutrients,finalFoodInputFoods,foodMeasures,foodAttributes,foodAttributeTypes,foodVersionIds,commonNames,additionalDescriptions,foodCode,foodCategoryId
0,2062724,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,041268198153,2021-10-28,Hannaford Bros. Co.,HANNAFORD,"ROASTED PEANUTS, SUGAR, CONTAINS 2% OR LESS OF...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
1,2062814,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,042187264981,2021-10-28,"S. M. Flickinger Co., Inc.",BEST YET,"ROASTED PEANUTS, CORN SYRUP, HYDROGENATED RAPE...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
2,2022842,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,725439941730,2021-10-28,"Delhaize America, Inc.",NATURE'S PLACE,"ORGANIC DRY ROASTED PEANUTS, SEA SALT.",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
3,2104256,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,075450173598,2021-10-28,"Hy-Vee, Inc.",THAT'S SMART!,"ROASTED PEANUTS, SUGAR, HYDROGENATED VEGETABLE...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
4,2096140,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,021110210219,2021-10-28,"Marsh Supermarkets, LLC",MARSH,"PEANUTS, SUGAR, CONTAINS 2% OR LESS OF: FULLY ...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
5,2118763,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,041270934428,2021-10-28,"Iga, Inc.",IGA,"SELECTED ROASTED PEANUTS, SUGAR, HYDROGENATED ...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
6,2127418,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,041497121960,2021-10-28,"Weis Markets, Inc.",WEIS QUALITY,"ROASTED PEANUTS, SUGAR, CONTAINS 2% OR LESS OF...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
7,1862235,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,070980134095,2021-07-29,Bostonia Beverage,CENTRELLA,"ROASTED PEANUTS, SUGAR, CONTAINS LESS THAN 2% ...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
8,1851674,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,036800442115,2021-07-29,"Topco Associates, Inc.",FOOD CLUB,"ROASTED PEANUTS, SUGAR, CONTAINS LESS THAN 2% ...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[],NaN,NaN,NaN,NaN
9,1505463,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,041303031100,2021-03-19,"Supervalu, Inc.",ESSENTIAL EVERYDAY,"ROASTED PEANUTS, SUGAR, CONTAINS 2% OR LESS OF...",United States,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],"[{'name': 'Update Log', 'description': 'Change...",[],NaN,NaN,NaN,NaN


In [14]:
diet_reqs = pd.read_csv("diet_reqs.csv")
diet_reqs

,Nutrition,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
0,Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
1,Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
2,"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
3,"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
4,"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
5,"Carbohydrate, by difference",RDA,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
6,"Iron, Fe",RDA,7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
7,"Magnesium, Mg",RDA,80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
8,Niacin,RDA,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0
9,"Phosphorus, P",RDA,460.0,500.0,500.0,1250.0,1250.0,1250.0,1250.0,700.0,700.0,700.0,700.0,700.0,700.0


In [17]:
import re 
#change this to take the data columns, clean them such that its only these
def adjust_groups(groups):
    valid_groups = []
    for g in groups:
        if g not in diet_reqs.columns:
            spl = re.split(" ", g)
            final_groups = []
            if spl[1] not in ages: 
                ranges = find_full_range(g)
            else: 
                final_group[0] = final_group[0] + spl[1]
                    #if spl[0] != "M" or spl[0] != "F":
                #default male 
                final_group = "M "
            #else:
                #final_group = spl[0] + " "
            #ranges = find_full_range(spl[1])
            
        else:
            valid_groups.append(g)
            
def find_full_range(g):
    start = []
    nums = re.split("-", g)
    start = nums[0]
    end = nums[1]
    found_start = False 
    
    
        
        
            

In [18]:
#Deliverable A: Creating Population Diet Requirement Function
groups = ['M 19-30','F 51+']
def create_pop(group,activity):
    group = np.append('Nutrition',group)
    pop_req = diet_reqs[group]
    if activity:
        pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300
    return pop_req
create_pop(groups, True)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


,Nutrition,M 19-30,F 51+
0,NaN,2700.0,1900.0
1,Protein,56.0,46.0
2,"Fiber, total dietary",33.6,22.4
3,"Folate, DFE",400.0,400.0
4,"Calcium, Ca",1000.0,1200.0
5,"Carbohydrate, by difference",130.0,130.0
6,"Iron, Fe",8.0,8.0
7,"Magnesium, Mg",400.0,320.0
8,Niacin,16.0,14.0
9,"Phosphorus, P",700.0,700.0


In [21]:
ingredients = pd.read_csv("trial-nutrition.csv")
ingredients

,Food Type,Size,fdcId,Brand,Energy,Protein,"Fiber, total dietary","Folate, DFE","Calcium, Ca","Carbohydrate, by difference",...,Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn","Sodium, Na",Type
0,Asparagus,100 g/1 hectogram,1956233.0,PERO FAMILY FARMS,18 kcal,2.35 g,2.4 g,52 µg,24 mg,3.53 g,...,0.143 mg,38 µg,0 µg,0.091 mg,5.9 mg,1.13 mg,41.6 µg,0.54 mg,2 mg,M
1,Avocado Chunks,100 g/1 hectogram,1102652.0,Survey (FNDDS),160 kcal,2 g,6.7 g,81 µg,12 mg,8.53 g,...,0.067 mg,7 µg,0 µg,0.257 mg,10 mg,2.07 mg,21 µg,0.64 mg,7 mg,M
2,Flavored Brown Rice,100 g/1 hectogram,1102470.0,Survey (FNDDS),106 kcal,2.76 g,1.2 g,10 µg,7 mg,19.3 g,...,0.071 mg,16 µg,0 µg,0.106 mg,0 mg,0.35 mg,2.4 µg,0.99 mg,293 mg,M
3,Carrots,100 g/1 hectogram,1871703.0,FULL CIRCLE MARKET,38 kcal,1.28 g,2.6 g,19 µg,38 mg,8.97 g,...,0.066 mg,835 µg,0 µg,0.138 mg,6.4 mg,0.66 mg,13.2 µg,0.24 mg,71 mg,M
4,Chicken,100 g/1 hectogram,2090362.0,"Wegmans Food Markets, Inc.",188 kcal,24.7 g,0 g,8 µg,0 mg,1.18 g,...,0.07 mg,8 µg,0.36 µg,0.246 mg,0 mg,0 mg,0 µg,1.6 mg,353 mg,M
5,Chickpeas,100 g/1 hectogram,1925585.0,13 Foods LLC,175 kcal,8.75 g,13.8 g,155 µg,50 mg,27.5 g,...,0.102,1 µg,0 µg,0.123 mg,1.5 mg,1.09 mg,11.4 µg,1.5 mg,11 mg,M
6,Greek Yogurt,100 g/1 hectogram,468388.0,Fresh & Easy,73 kcal,8 g,0 g,7 µg,100 mg,10 g,...,0.022 mg,1 µg,0.7 µg,0.064 mg,0 mg,0.01 mg,0 µg,0.53 mg,33 mg,M
7,Pear,100 g/1 hectogram,1102682.0,Survey (FNDDS),57 kcal,0.36 g,3.1 g,7 µg,9 mg,15.2 g,...,0.012 mg,1 µg,0 µg,0.029 mg,4.3 mg,0.12 mg,4.4 µg,0.1 mg,1 mg,M
8,White Potatoes,100 g/1 hectogram,1102879.0,Survey (FNDDS),125 kcal,1.87 g,1.4 g,9 µg,5 mg,20.4 g,...,0.1 mg,18 µg,0 µg,0.312 mg,12.1 mg,0.52 mg,4.1 µg,0.28 mg,167 mg,M
9,Raspberries,100 g/1 hectogram,1102708.0,Survey (FNDDS),52 kcal,1.2 g,6.5 g,21 µg,25 mg,11.9 g,...,0.032 mg,2 µg,0 µg,0.055 mg,26.2 mg,0.87 mg,7.8 µg,0.42 mg,1 mg,M


In [ ]:
api_price_key = 